In [13]:
import string
import nltk
import pandas as pd
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer

# nltk.download("stopwords")

import string
import re

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:20.4f}'.format

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import SGDClassifier

import numpy as np

from scipy import sparse

In [14]:
test = pd.read_csv("s3://advancedml-koch-mathur-hinkson/test.csv")

In [17]:
train = pd.read_csv("s3://advancedml-koch-mathur-hinkson/train.csv")

ReadTimeoutError: Read timeout on endpoint URL: "None"

In [ ]:
stops = set(stopwords.words('english'))
stops

In [ ]:
# intialize stemmer
ps = PorterStemmer() 
ls = LancasterStemmer()

# define stopwords
stops = set(stopwords.words('english'))
stops.add('')

approved_stop_words = {"not", "get", "against", "haven", "haven't","aren't", 
                       "aren", "should", "shouldn", "shouldn't", "themselves", 
                       "them", "under", "over", 'won', "won't", "wouldn'", 
                       "wouldn't"}

stops = stops - approved_stop_words

def clean_text(text, stop_ws=stops, stemmer=ps, str_output=True):
    
    t = text.replace("-", " ").split(" ")
    t = [w.strip(string.punctuation) for w in t]
    
    if stop_ws:
        t = [w.lower() for w in t if w not in stop_ws]
    
    if stemmer:
        t = [stemmer.stem(w) for w in t]
    
    if str_output:
        return ' '.join(t)
    else:
        return t
    

In [ ]:
train.columns

In [2]:
def add_text_cleaning_cols(df):
    
    df['split'] = df["comment_text"].apply(lambda x: x.split(" "))
    df['cleaned_w_stopwords_str'] = df["comment_text"].apply(clean_text,args=(None,None,True),)
    df['cleaned_w_stopwords'] = df["comment_text"].apply(clean_text,args=(None,None,False),)

    df['cleaned_no_stem_str'] = df["comment_text"].apply(clean_text,args=(None,stops,True),)
    df['cleaned_no_stem'] = df["comment_text"].apply(clean_text,args=(None,stops,False),)
    
    df['cleaned_porter_str'] = df["comment_text"].apply(clean_text,args=(stops,ps,True),)
    df['cleaned_porter'] = df["comment_text"].apply(clean_text,args=(stops,ps,False),)
    
    df['cleaned_lancaster_str'] = df["comment_text"].apply(clean_text,args=(stops,ls,True),)
    df['cleaned_lancaster'] = df["comment_text"].apply(clean_text,args=(stops,ls,False),)

    df['perc_upper'] = df["comment_text"].apply(lambda x: 0 if x == 0 else round((len(re.findall(r'[A-Z]',x)) / len(x)), 3))

    df['num_exclam'] = df["comment_text"].apply(lambda x:(len(re.findall(r'!',x))))
    
    df['num_words'] = df["split"].apply(lambda x: len(x))
    
    df['perc_stopwords'] = round((df.num_words - df['cleaned_no_stem'].apply(lambda x: 0 if df.num_words == 0 else len(x))) / df.num_words,3) 
    
    df['num_upper_words'] = df["split"].apply(lambda x: sum(map(str.isupper, x)) 
)
    

In [19]:
train['stemmed_comments'] = train.comment_text.apply(lambda x: clean_text(x, stops))

In [47]:
train.rename({'stemmed_comments': 'clean_porter'}, inplace=True, axis=1)

In [53]:
train['clean_lancaster'] = train.comment_text.apply(lambda x: clean_text(x, stops, ls))

In [54]:
train['clean_unstemmed'] = train.comment_text.apply(lambda x: clean_text(x, stops, False))

In [55]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,clean_porter,clean_lancaster,clean_unstemmed
0,59848,0.0000,"This is so cool. It's like, 'would you want yo...",0.0000,0.0000,0.0000,0.0000,0.0000,nan,nan,...,0,0,0,0,0.0000,0,4,"[thi, cool, it', like, would, want, mother, re...","[thi, cool, it's, lik, would, want, moth, read...","[this, cool, it's, like, would, want, mother, ..."
1,59849,0.0000,Thank you!! This would make my life a lot less...,0.0000,0.0000,0.0000,0.0000,0.0000,nan,nan,...,0,0,0,0,0.0000,0,4,"[thank, thi, would, make, life, lot, less, anx...","[thank, thi, would, mak, lif, lot, less, anxy,...","[thank, this, would, make, life, lot, less, an..."
2,59852,0.0000,This is such an urgent design problem; kudos t...,0.0000,0.0000,0.0000,0.0000,0.0000,nan,nan,...,0,0,0,0,0.0000,0,4,"[thi, urgent, design, problem, kudo, take, ver...","[thi, urg, design, problem, kudo, tak, very, i...","[this, urgent, design, problem, kudos, taking,..."
3,59855,0.0000,Is this something I'll be able to install on m...,0.0000,0.0000,0.0000,0.0000,0.0000,nan,nan,...,0,0,0,0,0.0000,0,4,"[is, someth, i'll, abl, instal, site, when, re...","[is, someth, i'll, abl, instal, sit, when, rel...","[is, something, i'll, able, install, site, whe..."
4,59856,0.8936,haha you guys are a bunch of losers.,0.0213,0.0000,0.0213,0.8723,0.0000,0.0000,0.0000,...,0,0,1,0,0.0000,4,47,"[haha, guy, bunch, loser]","[hah, guy, bunch, los]","[haha, guys, bunch, losers]"


In [56]:
train.to_pickle('train_stemmed.pkl')

In [21]:
test['stemmed_comments'] = test.comment_text.apply(lambda x: clean_text(x, stops))

In [49]:
test.rename({'stemmed_comments': 'clean_porter'}, inplace=True, axis=1)

In [51]:
test['clean_lancaster'] = test.comment_text.apply(lambda x: clean_text(x, stops, ls))

In [52]:
test['clean_unstemmed'] = test.comment_text.apply(lambda x: clean_text(x, stops, False))

In [57]:
test.head()

,id,comment_text,clean_porter,clean_lancaster,clean_unstemmed
0,7000000,Jeff Sessions is another one of Trump's Orwell...,"[jeff, session, anoth, one, trump', orwellian,...","[jeff, sess, anoth, on, trump's, orwel, cho, h...","[jeff, sessions, another, one, trump's, orwell..."
1,7000001,I actually inspected the infrastructure on Gra...,"[i, actual, inspect, infrastructur, grand, chi...","[i, act, inspect, infrastruct, grand, chief, s...","[i, actually, inspected, infrastructure, grand..."
2,7000002,No it won't . That's just wishful thinking on ...,"[no, won't, , that', wish, think, democrat, fa...","[no, won't, , that's, wish, think, democr, fau...","[no, won't, , that's, wishful, thinking, democ..."
3,7000003,Instead of wringing our hands and nibbling the...,"[instead, wring, hand, nibbl, peripheri, issu,...","[instead, wring, hand, nibbl, periphery, issu,...","[instead, wringing, hands, nibbling, periphery..."
4,7000004,how many of you commenters have garbage piled ...,"[mani, comment, garbag, pile, high, yard, bald...","[many, com, garb, pil, high, yard, bald, tir, ...","[many, commenters, garbage, piled, high, yard,..."


In [58]:
test.to_pickle('test_stemmed.pkl')

In [36]:
def make_ngrams(clean, n=2):
    grams = []
    
    for i in range(len(clean)-(n-1)):
        grams.append(' '.join(clean[i:i+n]))

    return grams

def ngrams(preprocessed, n):
    '''
    Covert a list of preprocessed strings into ngrams of length n.
    Should return X ngrams where X is equal to the length of the string less (n - 1).
    '''
    ngrams_tuples = []
    
    # ensure that all ngrams are of length n by specifying list position of 
    # first item in last ngram
    last_ngram_start = len(preprocessed) - (n - 1)     
    
    # for each string from position i through last ngram start position, create 
    # a tuple of length n
    for i in range(last_ngram_start):
        ngrams_tuples.append(tuple(preprocessed[i:i + n]))

    return ngrams_tuples

In [37]:
sentence = "Hi my name is Robert and I like wine and cheese, but only on Tuesday."

In [40]:
make_ngrams(cleaned_sentence, 5)

['hi name robert i like',
 'name robert i like wine',
 'robert i like wine chees',
 'i like wine chees tuesday']

In [ ]:
def find_top_k(items, k):
    '''
    Find the K most frequently occuring items
    Inputs:
        items: a list of items
        k: integer 
    Returns: sorted list of K tuples
    '''
    # create our dict and our holder for key/value tuples
    counter_tuples = []

    # iterate through the list, creating a dict key with value 1 if the item
    # isn't already represented, and incrementing value by 1 if it is
    ct_dict = iterate_dict(items)
        
    # iterate through the dict and append key/value pairs in tuple forms at a
    # list; sort tuples in list and return top k values
    for i, v in ct_dict.items():
        counter_tuples.append((i, v))

    tuples_sorted = sort_count_pairs(counter_tuples)

    top_k = tuples_sorted[:k]

    return top_k

In [ ]:
addtl_cols = ['perc_upper', 'num_exclam', 'num_words', 'perc_stopwords', 'num_upper_words']
#addtl_cols = ['perc_upper']

In [ ]:
train_sample.head()

In [ ]:
def run_model(model_df, train_perc=.80, addtl_feats =[''], model_type = "Gauss", 
              num_iter = 10, should_print=False, see_inside=False, comments="comment_text"):
    
    train_start = 0
    train_end = round(model_df.shape[0]*train_perc) 

    test_start = train_end
    test_end = model_df.shape[0]
    
    X_all = model_df[comments].values
    y_all = model_df['target'].values
    
    # tokenizing text
    count_vect = CountVectorizer()
    X_all_counts = count_vect.fit_transform(X_all.astype('U'))
    #print(X_all_counts.shape)

    # calculating frequencies
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_all_counts)
    X_all_tfidf = tf_transformer.transform(X_all_counts)

    print(X_all_tfidf.shape)
    
    if addtl_feats != ['']: # combine non-text and text features if necessary
        print("here")
        others_all = model_df[addtl_feats].values.reshape(-1,1)
        newfeatures_all = sparse.hstack((X_all_tfidf, others_all.astype(float))).tocsr()
    else:
        newfeatures_all = X_all_tfidf
    
    
    X_train = newfeatures_all[train_start:train_end]
    y_train = model_df[train_start:train_end]['target'].values
    y_train=y_train.astype('int')

    X_test = newfeatures_all[test_start:test_end]
    y_test = model_df[test_start:test_end]['target'].values
    
    
    if model_type == 'Multi':
        clf = MultinomialNB().fit(X_train, y_train)
    if model_type == "Gauss":
        clf = GaussianNB().fit(X_train, y_train) 
   
    predicted = clf.predict(X_test)
    accuracy = np.mean(predicted == y_test)

    if should_print == True:

        print("The accuracy on the test set is {}%.".format(round(accuracy*100,2)))    
    
    if see_inside == True:
        return clf, accuracy, X_all_counts, X_all_tfidf
    else:
        return clf, accuracy


In [ ]:
run_model(train_sample, should_print=True)

In [ ]:
for c in addtl_cols:
    print(c)
    print(run_model(train_sample, comments = comm, addtl_feats = [c], should_print=True))
    print()

In [ ]:
for comm in ['cleaned_w_stopwords', 'cleaned_no_stem', 'cleaned_porter',
       'cleaned_lancaster']:
    print(comm)
    print(run_model(train_sample, comments = comm, should_print=True))
    print()